In [1]:
import pandas as pd
import seaborn as sns
from pydataset import data
import numpy as np
import os

from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay

import prepare as p
import model as m
import explore as e
import acquire as a
from env import get_db_url

#turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = a.get_comet_data()

In [3]:
df.drop(columns=['DT','A1', 'A2', 'A3', 'n_obs_used', 'per_y','condition_code', 'ad'], inplace = True)

In [4]:
df

,full_name,e,q,i,om,w,tp_cal,class,data_arc,two_body
0,C/-146 P1,1.0000,0.430,71.00,330.00,261.00,-146-06-28.0,PAR,5.0,T
1,C/-43 K1,1.0000,0.220,110.00,170.00,17.00,-043-05-25.0,PAR,54.0,T
2,C/240 V1,1.0000,0.370,44.00,214.00,82.00,0240-11-10.0,PAR,39.0,T
3,C/390 Q1,1.0000,0.920,36.00,356.00,23.00,0390-09-05.0,PAR,26.0,T
4,C/400 F1,1.0000,0.210,32.00,38.00,47.00,0400-02-25.0,PAR,22.0,T
...,...,...,...,...,...,...,...,...,...,...
2218,C/2022 R3 (Leonard),1.0004,5.129,43.08,8.24,72.80,2023-03-04.7,HYP,59.0,NaN
2219,C/2022 R6 (PANSTARRS),1.0021,6.571,57.02,150.75,319.84,2025-08-26.1,HYP,77.0,NaN
2220,C/2022 S3 (PANSTARRS),1.0004,0.838,78.56,157.32,267.96,2023-01-21.6,HYP,36.0,NaN
2221,C/2022 S4 (Lemmon),1.0007,2.767,101.03,220.16,268.34,2024-07-18.0,HYP,92.0,NaN


In [5]:
df.isnull().sum()

full_name      0
e              0
q              0
i              0
om             0
w              0
tp_cal         0
class          0
data_arc       3
two_body     401
dtype: int64

In [6]:
df.describe()

,e,q,i,om,w,data_arc
count,2223.000000,2223.000000,2223.000000,2223.000000,2223.000000,2220.000000
mean,1.001504,0.732643,119.298691,145.968965,107.478489,135.008559
std,0.050051,1.618525,39.455099,141.994880,75.324552,401.988861
min,1.000000,0.001000,1.660000,0.000000,0.350000,0.000000
25%,1.000000,0.005000,96.075000,9.265000,71.910000,0.000000
50%,1.000000,0.007000,142.380000,80.270000,83.600000,1.000000
75%,1.000000,0.642000,144.460000,314.690000,97.805000,35.000000
max,3.356200,11.427000,176.480000,359.980000,359.970000,6067.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2223 entries, 0 to 2222
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   full_name  2223 non-null   object 
 1   e          2223 non-null   float64
 2   q          2223 non-null   float64
 3   i          2223 non-null   float64
 4   om         2223 non-null   float64
 5   w          2223 non-null   float64
 6   tp_cal     2223 non-null   object 
 7   class      2223 non-null   object 
 8   data_arc   2220 non-null   float64
 9   two_body   1822 non-null   object 
dtypes: float64(6), object(4)
memory usage: 173.8+ KB


In [8]:
df['tp_cal']=df['tp_cal'].str.split(pat='.').str[0]

In [9]:
df.head()

,full_name,e,q,i,om,w,tp_cal,class,data_arc,two_body
0,C/-146 P1,1.0,0.43,71.0,330.0,261.0,-146-06-28,PAR,5.0,T
1,C/-43 K1,1.0,0.22,110.0,170.0,17.0,-043-05-25,PAR,54.0,T
2,C/240 V1,1.0,0.37,44.0,214.0,82.0,0240-11-10,PAR,39.0,T
3,C/390 Q1,1.0,0.92,36.0,356.0,23.0,0390-09-05,PAR,26.0,T
4,C/400 F1,1.0,0.21,32.0,38.0,47.0,0400-02-25,PAR,22.0,T


In [12]:
df=df.drop(index=[0,1])

In [13]:
df.head()

,full_name,e,q,i,om,w,tp_cal,class,data_arc,two_body
2,C/240 V1,1.0,0.37,44.0,214.0,82.0,0240-11-10,PAR,39.0,T
3,C/390 Q1,1.0,0.92,36.0,356.0,23.0,0390-09-05,PAR,26.0,T
4,C/400 F1,1.0,0.21,32.0,38.0,47.0,0400-02-25,PAR,22.0,T
5,C/418 M1,1.0,0.35,110.0,310.0,240.0,0418-10-05,PAR,82.0,T
6,C/442 V1,1.0,1.53,106.0,271.0,178.0,0442-12-15,PAR,100.0,T


In [14]:
df=p.date_to_dt_index(df, 'tp_cal')

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 240-11-10 00:00:00